# CNN定点量化

本教程介绍了CNN定点量化以及量化的效应。本教程需要用到numpy的相关知识，如果不了解`numpy`可以产看此内容[poster](https://github.com/ringochu/TensorFlowDemo/blob/master/CheatSheets/Numpy_Python_Cheat_Sheet.pdf)

教程安排:

1. [定点或者Q表示](#定点或者Q表示)
2. [转换](#转换)
3. [算术运算](#算术运算)
4. [CNN和定点表达](#CNN和定点表达)
5. [练习](#练习)

### 教程目标:
1. 了解定点表示和Q表示
2. 使用定点进行数学运算
3. 了解它对实际应用的影响，比如：提速或降低精度

In [1]:
import numpy as np

## 定点或者Q表示

知识回顾：表示非整数或小数，开发人员通常有两个选项。第一种是使用浮点表示，它支持数值范围，精度和分辨率之间进行相互协调。名字给出了它主要属性，将整数部分和小数部分分开的点是浮动的，而不是固定的。然而，如果我们想要实现最佳性能，则这种数据类型及其算法在硬件中实现是具有挑战性的，除非处理设备具有足够的空间和资源用于专用浮点单元（FPU）。

这就是为什么在大多数可能需要恒定分辨率的低能耗，低性能的设备中，我们发现定点或者[Q](https://en.wikipedia.org/wiki/Q_(number_format))会比较实用，正如上文说到一样。然后这些位被假想的小数点分成两部分。第一部分是整数部分（IP），第二部分是分数部分。例如，假设我们总共操作16位，则Q16编号具有16个小数位; Q2.14号具有2个整数位和14个小数位。注意，为了表示带符号的数字，我们通常需要从整数部分再分配出来一个位来确定数字的符号。

定点数和浮点数的比较示意图: 

![fixed_float.png](../data/figs/fixed_float.png)
<h6><center>Figure 1. Fixed point representation and Floating point</center></h6>

这种表示方式既有优点也有缺点，一方面来说，它的底层设计是很容易实现的，这样设备可以提高性能同时降低功耗。另一方面，与浮点相比较，它可能会引入更多的计算错误。

我们可以来看看下面的例子。我们创建一个具有10个浮点数的数组，均匀分布在`1`和`2`之间。假设我们在定点表示中只允许`2`个小数位，我们将得到一个只有三个唯一值的数组：`1.0`,`1.5`和`2`。这个例子表明，粗心选择的分数位数（在这种情况下为2）可能会在转换后产生很大的错误。你还注意到，在将分数位数增加到`16`之后，错误变得不那么重要。

意到`a_fix_2`和`a_fix_16`仍然是浮点数阵列。他们的数据类型都是`numpy.float32`。当与他们计算时，我们需要将他们乘以比例因子， `2^f`（f是分数的位数），并转换他们的每一个元素。

我们将在以下部分中解释此转换的工作原理，以及在本教程的最后部分中讲解如何选择合适的小数位。

In [2]:
# create a floating-point array
a = np.linspace(1, 2, 10)

# scale the output as if was represented with only 2 fractional bits
a_fix_2 = np.round(a * 2) * (1.0 / 2)

# Fractional bits
a_fix_16 = np.round(a * 16) * (1.0 / 16)

print("Original array:", a)
print("2 fraction bits:", a_fix_2)
print("16 fraction bits:", a_fix_16)

Original array: [1.         1.11111111 1.22222222 1.33333333 1.44444444 1.55555556
 1.66666667 1.77777778 1.88888889 2.        ]
2 fraction bits: [1.  1.  1.  1.5 1.5 1.5 1.5 2.  2.  2. ]
16 fraction bits: [1.     1.125  1.25   1.3125 1.4375 1.5625 1.6875 1.75   1.875  2.    ]


### 二进制定点表示

在本教程中，我们将主要讨论动态定点表示，在课程中提出的另一个选项是二进制定点表示。这两者的区别在于，第一，MSB基本上代表负数。比如，你想用三位表示-3，在二进制中你只需要简单的用101表示，1作为符号位然后一共11或111。简单地说，数字的正号由第一个符号位得出，相关信息可以参考图一。 

## 转换

##### 浮点到Q

将数字从浮点转换到Qm.n格式需要: 
1. 将浮点数乘以2<sup>n</sup>, -- 这是将数字左移n个位置的移位
2. 舍入到最接近的整数

##### Q到浮点
1. 如果是整数则将数字转换成浮点数，否则删除二进制点。
2. 乘以2<sup>-n</sup>

为了在例子中展示这个过程，我们将以U8.8数字`2.1`为例。然后，我们将浮点数乘以小数位数，在这里`n=8`。之后进行第二步，将数字四舍五入之后，你可以看到第10位（固定点之后的第2位）是1，实际上表示2。最接近0.1的表达是小数点后为1的第4位，第5位以及第7位：2<sup>-4</sup> + 2<sup>-5</sup> + 2<sup>-7</sup> = 0.1015625。所以加上2正好等于2.1015625。你可以尝试任何数字来加强对此知识的理解。

In [3]:
# Given that we have UQ8.8 format (UQm.n) 
m = 8
n = 8
f = 2.1
q = f * 2**n
rounded = round(q)
print("The number in Q format is: {} and its binary representation is: {} and hex is: {}".format(rounded, bin(rounded), hex(rounded)))
print("The number back as a float is: {}".format(rounded*2**(-n)))

The number in Q format is: 538 and its binary representation is: 0b1000011010 and hex is: 0x21a
The number back as a float is: 2.1015625


这是这一表达存在的另一个问题，如果你正在进行数字的格式转换，那么数组的精确度会降低从而导致准确性的降低。请记住，当进行数值计算时，必须使输出数据的范围在选定的数据表达范围内。

## 算术运算

用定点数表达的一个大的好处在于：可以直接在ALU上进行算术运算。和上文的U8.8例子一样，我们可以用一些例子来演示它。我们将从加法运算和减法运算开始，想到我们可能遇到的唯一的问题，如果我们加上U1.1数字，例如01（0.5）和01（0.5），整数部分的变化的情况和一般加法一样。之后是减法。

在这个例子中，我们有a，b两个数字，这两个数字已经事先通过位移和舍入将其转换为定点表示。两者相加或者相减，然后再把他们转换成浮点数来看看他们的不同。

##### 加法 & 减法

In [4]:
i = 8
f = 8
a = 2.1
b = 2.3
a_fp = round(a * 2**f)
b_fp = round(b * 2**f)
c_fp = a_fp + b_fp
d_fp = a_fp - b_fp
print("The addition back as a float is: {}, using floats: {}".format(c_fp*2**(-f),a+b))
print("The subtraction back as a float is: {}, using floats: {}".format(d_fp*2**(-f),a-b))

The addition back as a float is: 4.40234375, using floats: 4.4
The subtraction back as a float is: -0.19921875, using floats: -0.19999999999999973


##### 乘法

乘法有点复杂，我们必须引入饱和的概念。为了防止溢出（这也可能发生在加法或者减法的情况下），我们必须将结果置于我们选择的数据表示范围之内。

我们再次用U8.8的表达作为例子。为了保持准确性，中间数的乘法（临时变量）和除法结果必须是精度的两倍，在转换回所需的Q数之前必须小心舍入中间结果。因此，我们首先将数字转换为定点表示，分别转换为`a_fp`和`b_fp`。 然后我们可以执行乘法并将结果存储在临时变量(temp)中。然后我们必须通过将2除以f：`temp / 2 ** f`的幂来将结果从双精度缩减回来，如果存在溢出，则结果会饱和。最后，我们将结果转换回浮点数以证明计算的正确性。

In [5]:
def saturate(x):
    if x>0xFFFF:
        return 0xFFFF
    elif x<0x0:
        return 0x0
    else:
        return x

i = 8
f = 8

a = 2.1
b = 2.3

a_fp = round(a * 2**f)
b_fp = round(b * 2**f)

temp = a_fp * b_fp

result = saturate(temp / 2**f)
print("The multiplication back as a float is: {}, using floats: {}".format(result*2**(-f),a*b))

The multiplication back as a float is: 4.835235595703125, using floats: 4.83


#### 除法

还是假设我们有相同的`a`和`b`。将他们转化成定点数并分别用 `a_fp` 和 `b_fp`表示。我们需要预先乘以结果。我们假设用s来表示：`(s * a) / (s * b)`，它实际等于`a / b`。但是我们实际上想得到的是`s*（x / y）`，所以我们需要先乘以s。然后，我们进行除法运算，最后将结果转换回浮点数进行演示。

In [6]:
i = 8
f = 8

a = 2.1
b = 2.3

a_fp = round(a * 2**f)
b_fp = round(b * 2**f)

# Pre-multiply by the base/scale (Upscale to Q24 so that the result will be eventually back in Q16 format) 
temp = a_fp << f;


result = temp / b_fp;
print("The division back as a float is: {}, using floats: {}".format(result*2**(-f),a/b))

The division back as a float is: 0.9134125636672326, using floats: 0.9130434782608696


有人可能会问，如果我们有不同Q表示的组合。 然后，我们必须以某种方式协调值，并舍入他们，要怎么做呢？

这一转换通常涉及右移操作以消除来自高阶Q表示的位，例如 Q16至Q8。在消除这些位之前，我们需要考虑到执行舍入操作。舍入的偏差会影响到整个结果的精确性。一个常用的方法是添加1到分数最高有效位的数。

我们用U4.4的表示来演示如何舍入到U2.2的表示。首先，我们将原始数据显示为定点数，然后我们改变分数位数`f = 2`。然后，我们将最高有效位添加到该数字，并保持其他数字不变。 最后，我们将两个数字都右移然后舍入并将它们转换回浮点数，以便我们可以看到效果。

In [7]:
#Q4.4
i = 4
f = 4
a = 1.1
a_fp = round(a * 2**f)
print("Hexadecimal fixed point representation: {}".format(hex(a_fp)))
print("Fixed point representation, as float: {}".format(a_fp*2**(-f)))

# Converting into Q4.2
# Rounding by adding one to the MSB
f = 2
t_fp = a_fp
a_fp+=0x02
a_fp>>=2
t_fp>>=2
print("Converted fixed point representation, as float: {}".format(a_fp*2**(-f)))
print("Truncated fixed point representation, as float: {}".format(t_fp*2**(-f)))

Hexadecimal fixed point representation: 0x12
Fixed point representation, as float: 1.125
Converted fixed point representation, as float: 1.25
Truncated fixed point representation, as float: 1.0


另一种方法是将数字舍入到最进的整数来表达这个数字。此方法给出了我们原始数字1.1的最接近的表示。

我们再次用Q4.4表示，我们首先将我们的比例计算为1` << f`，然后我们将其加到0.5以让它慢慢靠近最接近的整数，来代替舍入操作。 最后，我们将数字转换回浮点数以显示结果。

In [8]:
from math import floor

i = 4
f = 4
a = 1.1
scale = 1 << f
rounded_and_scaled = floor(a * scale + 0.5)
print("Round-to-Nearest, as a float: {}".format(rounded_and_scaled*2**(-f)))

Round-to-Nearest, as a float: 1.125


更高级的方法通常是genuine位数或者位数以及是否是奇偶数等等，更多的可以看此链接[link](http://en.wikipedia.org/wiki/Rounding)。

## CNN和定点表示

这些方法看似非常不精确，但十分有效，特别是在机器学习方面。在这里我们有许多中间结果，权重，偏差，其中低位宽数据可能会使学习速率提速([link](https://arxiv.org/abs/1712.05877))。如果我们考虑嵌入式系统和FPGA，尤其如此。这些系统和FPGA可能没有足够的计算和内存功能来支持浮点数。

准确性，速度，资源也是在选择Q格式还是其他正确选则需要仔细考虑的事情。我们来看一个例子。在之前的教程1和2中，我们读到了面部检测任务和SSD架构，它通过使用浮点来解决这个问题，但我们可以把它们替换成定点表示。

对于不了解SSD的人来说，它于2016年11月底发布，在物体检测任务的性能和精度方面达到了新的记录，在标准数据集（如PascalVOC和COCO）上以每秒59帧的速度得分超过74％mAP（平均精度）。它被称为Single Shot，因为它的定位和分类的任务是在网络的单个前向传递中完成的，而MultiBox是由Szegedy等人开发的四边界框回归技术的名称。这个网络令人兴奋，因为它很有前景。 特别是在嵌入式系统方面，因为它的计算开销非常低。 您可以在以下位置找到更多信息： [Paper](https://arxiv.org/abs/1512.02325)或 [Blog](https://towardsdatascience.com/understanding-ssd-multibox-real-time-object-detection-in-deep-learning-495ef744fab)文章。

In [9]:
from IPython.display import HTML, display
display(HTML("<table><tr><td>Original input data<img src='../data/figs/original.jpg'></td><td>SQ15.16<img src='../data/figs/fixed_32_16Q16.jpg'></td><td>SQ7.8<img src='../data/figs/fixed_16_8Q8.jpg'></td><td>SQ29.2<img src='../data/figs/fixed_32_30Q2.jpg'></td><td>SQ1.15<img src='../data/figs/fixed_16_2Q14.jpg'></td></tr></table>"))

Original input data,SQ15.16,SQ7.8,SQ29.2,SQ1.15


这些是SSD示例中使用SQ15.16和SQ7.8的结果，结果差异并不显着。 但是，如果使用不同的比例，例如：Q30.2和Q14.2，结果会失真。

在使用LeNet CNN架构的MNIST分类挑战中可以更详细地看到这一点。 小数部分和整数部分的设置可能非常精细，因为准确性对此非常敏感。请注意，在每个层中，权重输入和输出使用相同的定点方案。

![Flowchart.png](../data/figs/accuracy_vs_fb.png)

## 练习

让我们看一下数字的范围和它的分辨率，例如：在U（无符号）Q2.14和S（有符号，也就是整数部分的一位代表符号）Q8.8，你会如何计算它？

In [18]:
# Please replace the zeros with reasonable values
# In total we have 16 bits to operate on
# UQ2.14
i = 2
f = 14
print("UQ2.14")
print("Range is: {} to {}".format(0,0))
print("Resolution is: {}".format(0))

# SQ8.8
i = 8
f = 8
print("SQ8.8")
print("Range is: {} to {}".format(0,0))
print("Resolution is: {}".format(0))

UQ2.14
Range is: 0 to 0
Resolution is: 0
SQ8.8
Range is: 0 to 0
Resolution is: 0
